In [1]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from cugraph.dask.comms import comms
import cugraph.dask as dask_cugraph
import cugraph, dask_cudf

input_data_path = '../datasets/karate.csv'

cluster = LocalCUDACluster()
client = Client(cluster)
comms.initialize(p2p=True)

# Helper function to set the reader chunk size to automatically get one partition per GPU
chunksize = dask_cugraph.get_chunksize(input_data_path)

# Multi-GPU CSV reader
e_list = dask_cudf.read_csv(input_data_path,
        chunksize = chunksize,
        delimiter=' ',
        names=['src', 'dst'],
        dtype=['int32', 'int32'])

G = cugraph.DiGraph()
G.from_dask_cudf_edgelist(e_list, source='src', destination='dst')

# now run BFS
bfs_df = dask_cugraph.bfs(G, 1, depth_limit=0).compute()

# All done, clean up
comms.destroy()
client.close()
cluster.close()

bfs_df

2022-05-20 03:53:39,266 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,313 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,314 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,323 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,323 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,362 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,395 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-20 03:53:39,420 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/dataframe.py:1055: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future vers

{'resource_handle': <pylibcugraph.experimental.esourceHandle object at 0x7fb4f04a1880>, 'mg': <pylibcugraph.experimental.GGraph object at 0x7fb4f0533b80>, 'sources': Series([], Name: global_id, dtype: int32), 'weights': '0     1.0\n1     1.0\n2     1.0\n3     1.0\n4     1.0\n5     1.0\n6     1.0\n7     1.0\n8     1.0\n9     1.0\n10    1.0\n11    1.0\n12    1.0\n13    1.0\n14    1.0\nName: renumbered_src, dtype: float32', 'direction_optimizing': False, 'depth_limit': 0, 'return_predecessors': True, 'do_expensive_check': True, 'data': [    renumbered_src  renumbered_dst
0                6              18
1                5              18
2                1              18
3               13              18
4               13              19
5               13              21
6               13              22
7               14              18
8               17              18
9               30              18
10              30              22
11              31              19
12   

,distance,vertex,predecessor
0,2147483647,10,-1
1,2147483647,14,-1
2,2147483647,33,-1
3,2147483647,32,-1
4,0,1,-1
5,2147483647,29,-1
6,2147483647,16,-1
7,2147483647,8,-1
8,2147483647,27,-1
9,2147483647,5,-1


In [ ]:
comms.destroy()
client.close()
cluster.close()